# call libaries

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go



import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16, ResNet50  # Example pre-trained models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix
import itertools

import os
from glob import glob
import random


# read data

In [14]:
train_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
test_path = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [15]:
train_gen = ImageDataGenerator(rescale=1/255, validation_split=0.2)
val_gen = ImageDataGenerator(rescale=1/255)


In [16]:
train=train_gen.flow_from_directory(train_path,
                                    target_size=(224,224),color_mode='rgb',class_mode='categorical',
                                    batch_size=256,seed=0,subset='training',shuffle=True)
validation=train_gen.flow_from_directory(train_path,
                                    target_size=(224,224),color_mode='rgb',class_mode='categorical',
                                    batch_size=256,seed=0,subset='validation',shuffle=False)

Found 56251 images belonging to 38 classes.
Found 14044 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


# Build InceptionV3 model

In [63]:
from tensorflow.keras.applications import InceptionV3
img_height, img_width = 224, 224  
batch_size = 32

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in base_model.layers:
    layer.trainable = False

In [64]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(38, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [65]:
model.compile(
    loss="categorical_crossentropy", 
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy", "precision", "recall"]
)

In [49]:
base_model.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)

In [23]:
early_stopping=EarlyStopping(monitor='val_loss',patience=8,verbose=1, restore_best_weights=True)
checkpoints=ModelCheckpoint('resnet50.keras', monitor='val_loss', save_best_only=True,verbose=1)
reduceonplateau=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.001)
callback=[early_stopping,reduceonplateau,checkpoints]

In [67]:
history =model.fit(train,
                   epochs=15,
                   validation_data=validation,
                   callbacks=callback,
                   batch_size=128)


Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.

I0000 00:00:1730637571.814300     702 service.cc:145] XLA service 0x7f44940570f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730637571.814357     702 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730637571.814361     702 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1730637606.451055     702 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4777 - loss: 2.0238 - precision: 0.8562 - recall: 0.2830
Epoch 1: val_loss improved from inf to 0.60855, saving model to resnet50.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 285s 3s/step - accuracy: 0.4802 - loss: 2.0129 - precision: 0.8566 - recall: 0.2860 - val_accuracy: 0.8145 - val_loss: 0.6085 - val_precision: 0.9025 - val_recall: 0.7262 - learning_rate: 0.0010
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 612ms/step - accuracy: 0.8515 - loss: 0.5005 - precision: 0.9177 - recall: 0.7820
Epoch 2: val_loss improved from 0.60855 to 0.49318, saving model to resnet50.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.8514 - loss: 0.5003 - precision: 0.9176 - recall: 0.7821 - val_accuracy: 0.8466 - val_loss: 0.4932 - val_precision: 0.9033 - val_recall: 0.7919 - learning_rate: 0.0010
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 619ms/step - accuracy: 0.8906 - loss: 0.3658 - precision: 0.9363 - recall: 0.8435
Epoch 3: val_loss improved from 0.49318 

# Testing model

In [30]:
test_gen = ImageDataGenerator(rescale=1./255)
test =test_gen.flow_from_directory(test_path,
                                    target_size=(224,224),color_mode='rgb',class_mode='categorical',
                                    batch_size=256)

Found 17572 images belonging to 38 classes.


In [76]:
results = model.evaluate(test) 

550/550 ━━━━━━━━━━━━━━━━━━━━ 438s 760ms/step - accuracy: 0.9276 - loss: 0.2259 - precision: 0.9398 - recall: 0.9177


# Vgg16 model

In [18]:
# Load VGG16 model
img_height, img_width = 224, 224  
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pooling layer to reduce dimensions
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(38, activation='softmax')(x)  # Final output layer

model = Model(inputs=base_model.input, outputs=predictions)


In [19]:
model.compile(
    loss="categorical_crossentropy",          
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy", "precision", "recall"]
)

In [20]:
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [24]:
history =model.fit(train,
                   epochs=8,
                   validation_data=validation,
                   callbacks=callback,
                   batch_size=128)


Epoch 1/8


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1730647997.221600     107 service.cc:145] XLA service 0x7b828c001880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730647997.221656     107 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730647997.221660     107 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2024-11-03 15:33:31.066839: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=0,k4=1,k5=1,k6=0,k7=0,k19=0} for conv (f32[256,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[256,64,224,224]{3

68/69 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2579 - loss: 2.9941 - precision: 0.5865 - recall: 0.0202

2024-11-03 15:36:19.213557: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=2,k3=0} for conv (f32[164,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[164,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-11-03 15:36:19.368911: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.155461331s
Trying algorithm eng11{k2=2,k3=0} for conv (f32[164,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[164,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"op

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2603 - loss: 2.9844 - precision: 0.5914 - recall: 0.0212

2024-11-03 15:38:33.574904: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=3,k3=0} for conv (f32[220,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[220,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-11-03 15:38:33.889268: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.31453096s
Trying algorithm eng11{k2=3,k3=0} for conv (f32[220,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[220,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"ope


Epoch 1: val_loss improved from inf to 1.28288, saving model to resnet50.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 376s 4s/step - accuracy: 0.2627 - loss: 2.9750 - precision: 0.5962 - recall: 0.0221 - val_accuracy: 0.6530 - val_loss: 1.2829 - val_precision: 0.9246 - val_recall: 0.3238 - learning_rate: 0.0010
Epoch 2/8
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6914 - loss: 1.1033 - precision: 0.9061 - recall: 0.4293
Epoch 2: val_loss improved from 1.28288 to 0.81346, saving model to resnet50.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.6918 - loss: 1.1015 - precision: 0.9061 - recall: 0.4303 - val_accuracy: 0.7495 - val_loss: 0.8135 - val_precision: 0.8833 - val_recall: 0.6059 - learning_rate: 0.0010
Epoch 3/8
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7830 - loss: 0.7293 - precision: 0.9054 - recall: 0.6474
Epoch 3: val_loss improved from 0.81346 to 0.61240, saving model to resnet50.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.7832 - loss: 0.728

In [31]:
results = model.evaluate(test) 

69/69 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.8993 - loss: 0.3254 - precision: 0.9381 - recall: 0.8591
